Reddit user's gender classificator 


In [1]:
%pylab inline
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import stop_words
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import warnings
warnings.filterwarnings("ignore")

Populating the interactive namespace from numpy and matplotlib


/home/ale/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)
[nltk_data] Downloading package stopwords to /home/ale/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
train_data = pd.read_csv("/home/ale/Scrivania/reddit_chal/train_data.csv", encoding="utf8")
clean_train_data=train_data
train_data.head(10)

,author,subreddit,created_utc,body
0,Shamus_Aran,mylittlepony,1.388534e+09,I don't think we'd get nearly as much fanficti...
1,Riddance,sex,1.388534e+09,"Thanks. I made it up, that's how I got over my..."
2,Secret_Wizard,DragonsDogma,1.388534e+09,Are you sure you aren't confusing Cyclops (the...
3,Penultimatum,malefashionadvice,1.388534e+09,dont do this to me bro
4,7-SE7EN-7,todayilearned,1.388534e+09,That's what we do when we can't find a mate
5,Sahil17,Christianity,1.388534e+09,Damn I love this question. Here's what I think...
6,spyfer,NetflixBestOf,1.388534e+09,Sorry about that one post btw lol. I hope it d...
7,tehftw,AskReddit,1.388534e+09,Ad 1 &amp; 2. Right. I was mistaken.\n\nAd 3. ...
8,timsstuff,movies,1.388534e+09,I thought it was Colm Meaney whenever he's in ...
9,Bullseye4hire,hockey,1.388534e+09,FAck. Looks like 4 mins.


clean_train_data['word_length'] = clean_train_data['body'].map(lambda x: len(x.strip().split(' ')))

In [11]:

#Cleaning text
clean_train_data['body']=clean_train_data['body'].apply(lambda x: re.sub('\s+', ' ', x))
clean_train_data['body']=clean_train_data['body'].apply(lambda x: re.sub(r"http\S+", "", x))
clean_train_data['body']=clean_train_data['body'].apply(lambda x: re.sub(" \d+", " ", x))
clean_train_data['body']=clean_train_data['body'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
clean_train_data['body']=clean_train_data['body'].apply(lambda x: re.sub("\d+", " ", x))
clean_train_data['body']=clean_train_data['body'].apply(lambda x: re.sub('\[.*?\]', '', x))
clean_train_data['body']=clean_train_data['body'].apply(lambda x: re.sub('https?://\S+|www\.\S+', '',  x))
clean_train_data['body']=clean_train_data['body'].apply(lambda x: re.sub('<.*?>+', '',  x))
clean_train_data['body']=clean_train_data['body'].apply(lambda x: re.sub('\w*\d\w*', '', x))
clean_train_data['body']=clean_train_data['body'].apply(lambda x: re.sub('^a-zA-Z0-9 -', '', x))
clean_train_data.body = clean_train_data.body.str.replace("(<br/>)", "")
clean_train_data.body = clean_train_data.body.str.replace('(<a).*(>).*(</a>)', '')
clean_train_data.body = clean_train_data.body.str.replace('(&amp)', '')
clean_train_data.body = clean_train_data.body.str.replace('(&gt)', '')
clean_train_data.body = clean_train_data.body.str.replace('(&lt)', '')
clean_train_data.body = clean_train_data.body.str.replace('(\xa0)', '')  



#Making all text lowercase and getting rid of the date column
clean_train_data['body']=clean_train_data['body'].str.lower()
clean_train_data=clean_train_data.drop(['created_utc'], axis=1)

len(clean_train_data)

296042

In [44]:
#Removing stopwords taken both from the ntlk that gensim corpus (the appended ones)
from nltk.corpus import stopwords
stopwords=stopwords.words('english')

stopwords.extend(['know', 'youre', 'want','thats','ad', 'amp', 'id', 'dont','like', 'people','im', 'dont', 'one', 'think', 'would', 'during', 'among', 'thereafter', 'only', 'hers', 'in', 'none', 'with', 'un', 'put', 'hence', 'each', 'would', 'have', 'to', 'itself', 'that', 'seeming', 'hereupon', 'someone', 'eight', 'she', 'forty', 'much', 'throughout', 'less', 'was', 'interest', 'elsewhere', 'already', 'whatever', 'or', 'seem', 'fire', 'however', 'keep', 'detail', 'both', 'yourselves', 'indeed', 'enough', 'too', 'us', 'wherein', 'himself', 'behind', 'everything', 'part', 'made', 'thereupon', 'for', 'nor', 'before', 'front', 'sincere', 'really', 'than', 'alone', 'doing', 'amongst', 'across', 'him', 'another', 'some', 'whoever', 'four', 'other', 'latterly', 'off', 'sometime', 'above', 'often', 'herein', 'am', 'whereby', 'although', 'who', 'should', 'amount', 'anyway', 'else', 'upon', 'this', 'when', 'we', 'few', 'anywhere', 'will', 'though', 'being', 'fill', 'used', 'full', 'thru', 'call', 'whereafter', 'various', 'has', 'same', 'former', 'whereas', 'what', 'had', 'mostly', 'onto', 'go', 'could', 'yourself', 'meanwhile', 'beyond', 'beside', 'ours', 'side', 'our', 'five', 'nobody', 'herself', 'is', 'ever', 'they', 'here', 'eleven', 'fifty', 'therefore', 'nothing', 'not', 'mill', 'without', 'whence', 'get', 'whither', 'then', 'no', 'own', 'many', 'anything', 'etc', 'make', 'from', 'against', 'ltd', 'next', 'afterwards', 'unless', 'while', 'thin', 'beforehand', 'by', 'amoungst', 'you', 'third', 'as', 'those', 'done', 'becoming', 'say', 'either', 'doesn', 'twenty', 'his', 'yet', 'latter', 'somehow', 'are', 'these', 'mine', 'under', 'take', 'whose', 'others', 'over', 'perhaps', 'thence', 'does', 'where', 'two', 'always', 'your', 'wherever', 'became', 'which', 'about', 'but', 'towards', 'still', 'rather', 'quite', 'whether', 'somewhere', 'might', 'do', 'bottom', 'until', 'km', 'yours', 'serious', 'find', 'please', 'hasnt', 'otherwise', 'six', 'toward', 'sometimes', 'of', 'fifteen', 'eg', 'just', 'a', 'me', 'describe', 'why', 'an', 'and', 'may', 'within', 'kg', 'con', 're', 'nevertheless', 'through', 'very', 'anyhow', 'down', 'nowhere', 'now', 'it', 'cant', 'de', 'move', 'hereby', 'how', 'found', 'whom', 'were', 'together', 'again', 'moreover', 'first', 'never', 'below', 'between', 'computer', 'ten', 'into', 'see', 'everywhere', 'there', 'neither', 'every', 'couldnt', 'up', 'several', 'the', 'i', 'becomes', 'don', 'ie', 'been', 'whereupon', 'seemed', 'most', 'noone', 'whole', 'must', 'cannot', 'per', 'my', 'thereby', 'so', 'he', 'name', 'co', 'its', 'everyone', 'if', 'become', 'thick', 'thus', 'regarding', 'didn', 'give', 'all', 'show', 'any', 'using', 'on', 'further', 'around', 'back', 'least', 'since', 'anyone', 'once', 'can', 'bill', 'hereafter', 'be', 'seems', 'their', 'myself', 'nine', 'also', 'system', 'at', 'more', 'out', 'twelve', 'therein', 'almost', 'except', 'last', 'did', 'something', 'besides', 'via', 'whenever', 'formerly', 'cry', 'one', 'hundred', 'sixty', 'after', 'well', 'them', 'namely', 'empty', 'three', 'even', 'along', 'because', 'ourselves', 'such', 'top', 'due', 'inc', 'themselves'])

clean_train_data['body']=clean_train_data['body'].apply(lambda x: ' '.join([words for words in x.split() if words not in stopwords]))
clean_train_data.head(10)

,author,subreddit,body
0,Shamus_Aran,mylittlepony,wed nearly fanfiction pictures shipping banban...
1,Riddance,sex,thanks got heart break
2,Secret_Wizard,DragonsDogma,sure arent confusing cyclops easiest boss mons...
3,Penultimatum,malefashionadvice,bro
4,7-SE7EN-7,todayilearned,mate
5,Sahil17,Christianity,damn love question heres church fathers earlie...
6,spyfer,NetflixBestOf,sorry post btw lol hope didnt cause guys troub...
7,tehftw,AskReddit,right mistaken firearms access weapons
8,timsstuff,movies,thought colm meaney hes role colm meaney john ...
9,Bullseye4hire,hockey,fack looks mins


In [45]:
#Lemmatizer and stemmer do not seems to make a real difference

#from nltk.stem import WordNetLemmatizer 
#from nltk import punkt
#nltk.download('wordnet')
#nltk.download('punkt')
  
#lemmatizer = WordNetLemmatizer() 
#clean_train_data['body']=clean_train_data['body'].apply(lambda x: ' '.join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(x)]))


#from nltk.stem import LancasterStemmer
#stemmer=LancasterStemmer()

#clean_train_data['body']=clean_train_data['body'].apply(lambda x: ' '.join([stemmer.stem(w) for w in nltk.word_tokenize(x)]))




In [46]:
clean_train_data.head(10)

,author,subreddit,body
0,Shamus_Aran,mylittlepony,wed nearly fanfiction pictures shipping banban...
1,Riddance,sex,thanks got heart break
2,Secret_Wizard,DragonsDogma,sure arent confusing cyclops easiest boss mons...
3,Penultimatum,malefashionadvice,bro
4,7-SE7EN-7,todayilearned,mate
5,Sahil17,Christianity,damn love question heres church fathers earlie...
6,spyfer,NetflixBestOf,sorry post btw lol hope didnt cause guys troub...
7,tehftw,AskReddit,right mistaken firearms access weapons
8,timsstuff,movies,thought colm meaney hes role colm meaney john ...
9,Bullseye4hire,hockey,fack looks mins


In [47]:
target = pd.read_csv("/home/ale/Scrivania/reddit_chal/train_target.csv")

In [48]:
#Extract the subreddit list and mapping it to integers
subreddits = clean_train_data.subreddit.unique()
subreddits_map = pd.Series(index=subreddits, data=arange(subreddits.shape[0]))

print(subreddits_map)

mylittlepony            0
sex                     1
DragonsDogma            2
malefashionadvice       3
todayilearned           4
                     ... 
palegirls            3463
onions               3464
mumfordandsons       3465
infertility          3466
HangoutFest          3467
Length: 3468, dtype: int64


In [49]:
from scipy import sparse

In [50]:
def extract_features(group):
    group_subreddits = group['subreddit'].values 
    #print(group_subreddits.type)
    idxs = subreddits_map[group_subreddits].values 
    v = sparse.dok_matrix((1, subreddits.shape[0]))
  
    
    for idx in idxs:
        if not np.isnan(idx):
            v[0, idx] = 1
    
 
    return v.tocsr()




In [51]:
features_dict = {}

for author, group in clean_train_data.groupby('author'):
   
    features_dict[author] = extract_features(group)
    



In [52]:
X = sparse.vstack([features_dict[author] for author in target.author])
X

<5000x3468 sparse matrix of type '<class 'numpy.float64'>'
	with 49152 stored elements in Compressed Sparse Row format>

In [53]:
y = target.gender

In [54]:
def extract_text(group):
    group_text = group['body'].values
    return " ".join(group_text)

def extract_subrteddit_text(group):
    group_text = group['subreddit'].values
    return " ".join(group_text)

In [55]:
text_dict = {}

for author, group in clean_train_data.groupby('author'):
    text_dict[author] = extract_text(group)

In [56]:
author_text = [text_dict[author] for author in target.author]

In [57]:
subreddit_dict ={}
author_dict = {}

for subreddit, group in clean_train_data.groupby('subreddit'):
    subreddit_dict[subreddit] = extract_text(group)

    
for author, group in clean_train_data.groupby('author'):
    author_dict[author] = extract_text(group)
    

In [58]:
#Creating list of words taken from the corpus of top subreddit for men and women
#sources: https://www.techlazy.com/best-subreddits-for-men/
#http://redditlist.com/


men_list=[]
men_list.extend(subreddit_dict['malefashionadvice'].split(' '))
men_list.extend(subreddit_dict['MensRights'].split(' '))
men_list.extend(subreddit_dict['malehairadvice'].split(' '))
men_list.extend(subreddit_dict['beards'].split(' '))
men_list.extend(subreddit_dict['AskMen'].split(' '))
men_list.extend(subreddit_dict['AskMenOver30'].split(' '))


women_list=[]
women_list.extend(subreddit_dict['AskWomen'].split(' '))
women_list.extend(subreddit_dict['Feminism'].split(' '))
women_list.extend(subreddit_dict['TheGirlSurvivalGuide'].split(' '))
women_list.extend(subreddit_dict['GirlGamers'].split(' '))
women_list.extend(subreddit_dict['BodyAcceptance'].split(' '))
women_list.extend(subreddit_dict['AskWomenOver30'].split(' '))




print(len(women_list), len(men_list))

172269 176730


In [59]:
#Function to sort the wordlists by number of appearences and eliminate duplicates

from collections import Counter

def sort_unique(lst, n):
    result = (item for items, c in Counter(lst).most_common() for item in [items] * c)
    unique_list = list(dict.fromkeys(result))
    if len (lst)> n:
            unique_list = unique_list[:n]
    else: 
            unique_list=unique_list[:len(lst)]
    return unique_list

    


In [60]:
men_sorted_unique=  np.array(sort_unique(subreddit_dict['AskMen'].split(' '), 5000))
women_sorted_unique= np.array(sort_unique(subreddit_dict['AskWomen'].split(' '), 5000))

M_subreddit_gender_map = pd.Series(index=men_sorted_unique, data=arange(men_sorted_unique.shape[0]))
F_subreddit_gender_map = pd.Series(index=women_sorted_unique, data=arange(women_sorted_unique.shape[0]))


print(M_subreddit_gender_map)



time              0
women             1
good              2
way               3
ive               4
               ... 
prospects      4995
carrying       4996
furthermore    4997
damages        4998
approve        4999
Length: 5000, dtype: int64


In [61]:
#Function to create sparse matrices relatives to the m/f features

def extract_Mgender_features(group):

    #group_subreddits = group['subreddit'].values #raggruppa per subreddits (str) 
    group_text = group['body'].values
                        
    idxs = M_subreddit_gender_map[group_text].values

    
    #print(idxs)
    v = sparse.dok_matrix((1, men_sorted_unique.shape[0]))
    #print(v)
    
    for idx in idxs:
        if not np.isnan(idx):
            v[0, idx] =1
        
    
    
    return v.tocsr()


def extract_Fgender_features(group):

    #group_subreddits = group['subreddit'].values #raggruppa per subreddits (str) 
    group_text = group['body'].values
                        
    idxs = F_subreddit_gender_map[group_text].values
    #print(idxs)
    
    #print(idxs)
    v = sparse.dok_matrix((1, women_sorted_unique.shape[0]))
    #print(v)
    
    for idx in idxs:
        if not np.isnan(idx):
            v[0, idx] +=1
        
    
    
    return v.tocsr()




In [62]:
#Creating the m/dictionaries

M_features_dict = {}

for author, group in clean_train_data.groupby('author'):
   
    M_features_dict[author] = extract_Mgender_features(group)
    
    
F_features_dict = {}

for author, group in clean_train_data.groupby('author'):
   
    F_features_dict[author] = extract_Fgender_features(group)

In [63]:
#Create sparse matrices for m/f contributions
Xf = sparse.vstack([F_features_dict[author] for author in target.author])
Xm = sparse.vstack([M_features_dict[author] for author in target.author])

In [37]:
#Stacking all the sparse matrices
X=sparse.hstack([X, Xm, Xf]).tocsr()
X

<5000x13468 sparse matrix of type '<class 'numpy.float64'>'
	with 73879 stored elements in Compressed Sparse Row format>

In [89]:
from sklearn.metrics import accuracy_score, recall_score, precision_score ,f1_score,roc_auc_score

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

Form now on i'll try some different classifiers and choose the most performant.

In [90]:
##Model 
from sklearn.linear_model import LogisticRegression
#

clf_lr = LogisticRegression(C=5, max_iter=100)
clf_lr.fit(X, y)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1)
clf_lr.fit(X_train, y_train)
y_pred_lr = clf_lr.predict(X_test)

In [91]:
from sklearn.metrics import classification_report
print(classification_report(y_true=y_test, y_pred=y_pred_lr))

              precision    recall  f1-score   support

           0       0.83      0.93      0.88       350
           1       0.77      0.55      0.64       150

    accuracy                           0.82       500
   macro avg       0.80      0.74      0.76       500
weighted avg       0.81      0.82      0.81       500



In [92]:
confusion_matrix(y_true=y_test, y_pred=y_pred_lr)


array([[326,  24],
       [ 68,  82]])

In [102]:
from sklearn.model_selection import KFold, cross_val_predict, cross_val_score
kfold_lr = KFold(n_splits=10)
result_lr = cross_val_predict(estimator=clf_lr,X=X,y=y,cv=kfold_lr)


f1_lr=f1_score(y, result_lr)

f1_lr

0.68317247167436

In [103]:
acc_lr = accuracy_score(y, result_lr)
acc_lr

0.849

In [104]:
roc_auc_score_lr = roc_auc_score(y, y_score=result_lr)


Support vector machine

In [105]:
##Model
from sklearn.svm import SVC
clf_svc = SVC(kernel='linear', probability=True)
clf_svc.fit(X_train, y_train)

SVC(kernel='linear', probability=True)

In [106]:
y_pred_svc = clf_svc.predict(X_test)
print(accuracy_score(y_test, y_pred_svc))

0.818


In [107]:
print(classification_report(y_true=y_test, y_pred=y_pred_svc))

              precision    recall  f1-score   support

           0       0.82      0.95      0.88       350
           1       0.82      0.51      0.63       150

    accuracy                           0.82       500
   macro avg       0.82      0.73      0.75       500
weighted avg       0.82      0.82      0.80       500



In [108]:
confusion_matrix(y_true=y_test, y_pred=y_pred_svc)

array([[333,  17],
       [ 74,  76]])

In [109]:
kfold_svc = KFold(n_splits=10)
result_svc = cross_val_predict(estimator=clf_svc,X=X,y=y,cv=kfold_svc)


In [110]:
f1_svc=f1_score(y, result_svc)
f1_svc

0.663002114164905

In [111]:
acc_svc = accuracy_score(y, result_svc)
acc_svc

0.8406

In [112]:
roc_auc_score_svc = roc_auc_score(y, y_score=result_svc)
roc_auc_score_svc

0.7588135017488633

In [113]:
cross_val_svc = cross_val_score(SVC(kernel='linear', probability=True),X,y, cv=5, scoring='accuracy')
print (cross_val_svc)
print (cross_val_svc.mean())

[0.828 0.858 0.828 0.845 0.843]
0.8404


Random Forest



In [118]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import GridSearchCV
clf_1 = RandomForestClassifier()
Cs_1 = [10, 500]
Cs_2 = [None, 50, 400]
grid_1 = GridSearchCV(clf_1, param_grid={'n_estimators': Cs_1,'max_depth': Cs_2}, cv=5)
grid_1.fit(X_train, y_train)
print ("best parameter choice:", grid_1.best_params_)

best parameter choice: {'max_depth': 50, 'n_estimators': 500}


In [119]:
clf_rf = RandomForestClassifier(n_estimators=grid_1.best_params_['n_estimators'], max_depth=grid_1.best_params_['max_depth'])

clf_rf.fit(X_train, y_train)


RandomForestClassifier(max_depth=50, n_estimators=500)

In [120]:
y_pred_rf = clf_rf.predict(X_test)
print(accuracy_score(y_test, y_pred_rf))

0.798


In [121]:
print(classification_report(y_true=y_test, y_pred=y_pred_rf))

              precision    recall  f1-score   support

           0       0.78      0.99      0.87       350
           1       0.92      0.36      0.52       150

    accuracy                           0.80       500
   macro avg       0.85      0.67      0.69       500
weighted avg       0.82      0.80      0.77       500



In [122]:
confusion_matrix(y_true=y_test, y_pred=y_pred_rf)

array([[345,   5],
       [ 96,  54]])

In [123]:
kfold_rf = KFold(n_splits=5, random_state=42)
result_rf = cross_val_predict(estimator=clf_rf,X=X,y=y,cv=kfold_rf)
f1_rf=f1_score(y, result_rf)
f1_rf

0.5952976488244123

In [124]:
acc_rf = accuracy_score(y, result_rf)
acc_rf

0.8382

In [125]:
roc_auc_score_rf = roc_auc_score(y, y_score=result_rf)
roc_auc_score_rf 

0.7130015457243454

In [126]:
MultiNB=MultinomialNB()

In [127]:
MultiNB.fit(X_train,y_train)

MultinomialNB()

In [128]:
y_pred_MultiNB = MultiNB.predict(X_test)



In [129]:
print(accuracy_score(y_test, y_pred_MultiNB))

0.85


In [130]:
print(classification_report(y_true=y_test, y_pred=y_pred_MultiNB))

              precision    recall  f1-score   support

           0       0.85      0.96      0.90       350
           1       0.86      0.59      0.70       150

    accuracy                           0.85       500
   macro avg       0.86      0.78      0.80       500
weighted avg       0.85      0.85      0.84       500



In [131]:
confusion_matrix(y_true=y_test, y_pred=y_pred_MultiNB)

array([[336,  14],
       [ 61,  89]])

In [132]:
kfold_NB = KFold(n_splits=10)
result_NB = cross_val_predict(estimator=MultiNB,X=X,y=y,cv=kfold_NB)

In [133]:
acc_NB = accuracy_score(y, result_NB)
acc_NB

0.861

In [134]:
roc_auc_score_NB = roc_auc_score(y, y_score=result_NB)
roc_auc_score_NB 

0.7912442725664486

In [135]:
classifiers = ['Logistic Regression', 'Random Forest Classifier', 'SVC', 'MultiNB']
acc = np.array([acc_lr, acc_rf, acc_svc, acc_NB])
max_acc = np.argmax(acc)
rocauc = np.array([roc_auc_score_lr, roc_auc_score_rf, roc_auc_score_svc, roc_auc_score_NB])
max_rocauc = np.argmax(rocauc)
print (classifiers[max_acc] + ' is the best classifier for this problem')

MultiNB is the best classifier for this problem


In [136]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=.01).fit(X_train, y_train)

In [137]:
alpha = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
param_grid = dict(alpha=alpha)
grid = GridSearchCV(estimator=ridge, param_grid=param_grid, scoring='r2', verbose=1, n_jobs=4)
grid_result = grid.fit(X_train, y_train)
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Best Score:  0.3161325289447928
Best Params:  {'alpha': 10}


[Parallel(n_jobs=4)]: Done  28 out of  35 | elapsed:    0.9s remaining:    0.2s
[Parallel(n_jobs=4)]: Done  35 out of  35 | elapsed:    0.9s finished


In [ ]:
from sklearn.linear_model import ElasticNet
# Train model with default alpha=1 and l1_ratio=0.5
elastic_net = ElasticNet(alpha=1, l1_ratio=0.5).fit(X_train, y_train)
# get cross val scores

def get_cv_scores(model):
    scores = cross_val_score(model,
                             X_train,
                             y_train,
                             cv=5,
                             scoring='r2')
    
    print('CV Mean: ', np.mean(scores))
    print('STD: ', np.std(scores))
    print('\n')


get_cv_scores(elastic_net)


# find optimal alpha with grid search
alpha = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
l1_ratio = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
param_grid = dict(alpha=alpha, l1_ratio=l1_ratio)
grid = GridSearchCV(estimator=elastic_net, param_grid=param_grid, scoring='r2', verbose=1, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)



CV Mean:  -0.0013330575744815222
STD:  0.001458637859018578


Fitting 5 folds for each of 77 candidates, totalling 385 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.1min


In [ ]:
pipe = Pipeline([('classifier' , RandomForestClassifier())])
# pipe = Pipeline([('classifier', RandomForestClassifier())])

# Create param grid.

param_grid = [
    {'classifier' : [LogisticRegression()],
     'classifier__penalty' : ['l1', 'l2'],
    'classifier__C' : np.logspace(-4, 4, 20),
    'classifier__solver' : ['liblinear']},
    {'classifier' : [RandomForestClassifier()],
    'classifier__n_estimators' : list(range(10,101,10)),
    'classifier__max_features' : list(range(6,32,5))}
]

# Create grid search object

clf = GridSearchCV(pipe, param_grid = param_grid, cv = 5, verbose=True, n_jobs=-1)

# Fit on data

best_clf = clf.fit(X_train, y_train)